In [1]:
# !pip install tweepy
# !pip install emoji

In [2]:
import sys
import os
import glob
import tweepy as tw
from emoji import emojize
import pandas as pd

In [3]:
consumer_key = os.getenv('Twitter_Consumer_Key')
consumer_secret = os.getenv("Twitter_Consumer_Secret")
access_token = os.getenv('Twitter_Access_Token')
access_token_secret = os.getenv('Twitter_Access_Secret')

In [4]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True)
print(api.verify_credentials().name)
print('Successfully connected to the Twitter API.')

Ryearwood
Successfully connected to the Twitter API.


### Query Specification & Execution

In [144]:
query = '#afraid'
max_requests = 180

In [134]:
q = emojize(query) + ' -filter:retweets'
searched_tweets = []
last_id = -1
request_count = 0
while request_count < max_requests:
    try:
        new_tweets = api.search_tweets(q=q,
                                       lang='en',
                                       count=100,
                                       max_id=str(last_id - 1),
                                       tweet_mode='extended')
        if not new_tweets:
            break
        searched_tweets.extend(new_tweets)
        last_id = new_tweets[-1].id
        request_count += 1
    except tw.TweepError as error:
        print(error)
        break

In [135]:
data = []
for tweet in searched_tweets:
    data.append([tweet.id, tweet.created_at, tweet.user.screen_name, tweet.full_text])
df = pd.DataFrame(data=data, columns=['id', 'date', 'user', 'text'])
print(str(len(data)) + ' ' + query + ' tweets')

1716 #fear tweets


In [ ]:
# Map twitter emojis to text string
angry_emotion_relations = [":pouting_face:",":face_with_steam_from_nose:",":face_with_symbols_on_mouth:","#pissed","#furious","#rage"]
angry_set = set(angry_emotion_relations)

sad_emotion_relations = [":crying_face:","#sad","#depressed",":pensive_face:",":loudly_crying_face:",":broken_heart:"]
sad_set = set(sad_emotion_relations)

fear_emotion_relations = [":face_screaming_in_fear:",":fearful_face:","#anxious","#fear","#afraid","#scared","#scary",":anxious_face_with_sweat:","#worried"]
fear_set = set(fear_emotion_relations)

surprise_emotion_relations = [":flushed_face:",":thinking_face:",":face_with_rolling_eyes:",":exploding_head:", "#surprise","#excited"]
surprise_set = set(surprise_emotion_relations)

if query in angry_set:
    query = "#angry"
elif query in sad_set:
    query = "#sad"
elif query in fear_set:
    query = "#fear"
elif query in surprise_set:
    query = "#surprise"

filename = query + '.csv'
save_location = "C:/Users/russe/Desktop/LDA_Topic_Modeling/data/Datasets/Emotion_Data/"+filename
df.to_csv(save_location, mode='a', index=None,header=False)
print(f"Completed and data added to file location at {save_location}")

data = pd.read_csv(save_location)
print(f"Current CSV Length:{data.shape[0]}")

## Label & Compile Tweets

In [ ]:
%run ./Compile_Tweets.ipynb